# Local

In [ ]:
#!pip install playsound==1.2.2

In [ ]:
import cv2
import time
from playsound import playsound
import numpy as np
import torch
from tensorflow import keras
from PIL import Image
from keras.applications.resnet_v2 import preprocess_input
from datetime import datetime
from keras.utils import img_to_array
from keras.models import load_model
from datetime import datetime

## Loading models

In [ ]:
# YOLO
yolo_model = torch.hub.load('WongKinYiu/yolov7','custom','../object_detection/pt/0213_yolov7_epochs55.pt')

In [ ]:
# YOLO settings
yolo_model.conf=0.7
yolo_model.classes=27
yolo_model.iou=0.6

In [ ]:
# CNN
new_model = load_model("../binary_classification/models/0217_aug_flip")
idx_to_classes = {0: 'with-leash', 1: 'without-leash'}

No training configuration found in save file, so the model was *not* compiled. Compile it manually.


## FUNCTIONS

In [ ]:
def pred(img_path,probs=False):
    #print('-'*20)
    #Image.open(img_path).convert('RGB').resize((224,224)).show()
    img = keras.preprocessing.image.load_img(img_path, target_size=(224,224))
    array = keras.preprocessing.image.img_to_array(img) #(224,224,3)
    array = np.expand_dims(array, axis=0)  #(1, 224, 224, 3)
    img_array = preprocess_input(array)
    preds = new_model.predict(img_array)
    if probs:
        print('with vs without :',preds)
    return idx_to_classes[np.argmax(preds)]

In [ ]:
def system(IMAGE_FILE):
    results=yolo_model(IMAGE_FILE)
    pos_lists = results.pandas().xyxy
    pos_df = pos_lists[0]
    dogs=pos_df[['xmin', 'ymin', 'xmax', 'ymax']]
    confidences=pos_df['confidence']
    for i in range(len(dogs)):
        dog=dogs.iloc[i].tolist()
        dog=list(map(int,dog))
        conf=confidences.iloc[i]
        xmin,ymin,xmax,ymax=dog

        frame=Image.open(IMAGE_FILE).convert('RGB')
        name=IMAGE_FILE.split('.')[1].split('/')[-1]
        margin=20
        crop=frame.crop((max(xmin-margin,0),max(ymin-margin,0),min(xmax+margin,frame.width),min(ymax+margin,frame.height)))

        crop.save(name+'_crop.jpg')
        pred(name+'_crop.jpg',probs=True)

## REAL

In [ ]:
system('../test_images/jin_with.jpg')

1/1 [==============================] - 0s 126ms/step
with vs without : [[    0.99954  0.00046251]]


In [ ]:
system('../test_images/jin_without_noclothes.jpg')

1/1 [==============================] - 0s 111ms/step
with vs without : [[  0.0082826     0.99172]]


In [ ]:
system('../test_images/jin_without.jpg')

1/1 [==============================] - 0s 123ms/step
with vs without : [[  0.0010069     0.99899]]


# System ver

In [ ]:
###################### environments #############################
import cv2
import time
from playsound import playsound
import numpy as np
import torch
from tensorflow import keras
from PIL import Image
from keras.applications.resnet_v2 import preprocess_input
from keras.utils import img_to_array
from keras.models import load_model
from datetime import datetime

In [ ]:
################### YOLO model ##########################
print('yolo-start')
playsound("./sound/yolo.wav")
yolo_model = torch.hub.load('WongKinYiu/yolov7','custom','../object_detection/pt/0213_yolov7_epochs55.pt')
yolo_model.conf=0.7 # threshold
yolo_model.classes=27
yolo_model.iou=0.6

In [ ]:
################### CNN model ############################
print('cnn-start')
playsound("./sound/cnn.wav")
model = load_model("./0217_aug_flip")
idx_to_classes = {0: 'with-leash', 1: 'without-leash'}

cnn-start


No training configuration found in save file, so the model was *not* compiled. Compile it manually.


In [ ]:
def pred(img_path,probs=False):
    img = keras.preprocessing.image.load_img(img_path, target_size=(224,224))
    array = keras.preprocessing.image.img_to_array(img) #(224,224,3)
    array = np.expand_dims(array, axis=0)  #(1, 224, 224, 3)
    img_array = preprocess_input(array)
    preds = model.predict(img_array)
    if probs:
        print('with vs without :',preds)
    return idx_to_classes[np.argmax(preds)]

In [ ]:
def capture():   
    while True:
        webcam=cv2.VideoCapture(0)
        while True:
            ret,frame=webcam.read()
            results=yolo_model(frame,size=224)
            pos_lists = results.pandas().xyxy
            pos_df = pos_lists[0]
            dogs=pos_df[['xmin', 'ymin', 'xmax', 'ymax']]
            confidences=pos_df['confidence']
            if len(dogs)>=1:
                break
        for i in range(len(dogs)):
            dog=dogs.iloc[i].tolist()
            dog=list(map(int,dog))
            conf=confidences.iloc[i]
            
            now=datetime.now()
            name = now.strftime('%y%m%d%H%M%S') + '.jpg'
            xmin,ymin,xmax,ymax=dog
            margin=20
            height=frame.shape[0]
            width=frame.shape[1]
            cv2.imwrite(name,frame[max(ymin-margin,0):min(ymax+margin,height),max(xmin-margin,0):min(xmax+margin,width),:])
            ret=pred(name,probs=True)
            if(ret=='without-leash'):
                playsound('./sound/alert_final.wav')
        webcam.release()
        time.sleep(1)
    webcam.release()

In [ ]:
playsound("./sound/execute.wav")
capture()